# GNN con TODOS los Biomarcadores + Early Stopping + Regularización
Este notebook usa **TODOS los pacientes (6,210 visitas)** con regularización mejorada:

**Mejoras de regularización:**
- ⏸️ **Early Stopping**: Parada automática entre épocas 20-25 si no mejora validación
- 🎲 **Dropout aumentado**: 0.5 para evitar overfitting
- ⚖️ **L2 Regularization**: Weight decay = 1e-3 (10x más fuerte)
- 📊 **Monitoreo**: Tracking de overfitting y convergencia

**Estrategia LEFT JOIN:**
- Empezamos con TODAS las visitas demográficas
- Añadimos CSF, PET, MRI cuando están disponibles
- Cuando faltan: usamos 0 + añadimos indicadores binarios (HAS_CSF, HAS_PET, HAS_MRI)
- El GNN aprende a usar biomarcadores cuando están disponibles

**Features (~30):**
- Demographics: AGE, EDUCATION, GENDER, MARITAL_STATUS
- CSF: ABETA42, TAU, PTAU + ratios (cuando disponible)
- PET: CENTILOIDS, SUMMARY_SUVR (cuando disponible)
- MRI: 7 volúmenes cerebrales (cuando disponible)
- Missing indicators: HAS_CSF, HAS_PET, HAS_MRI

**Ventaja**: Máximo dataset + regularización óptima = Mejor generalización

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

torch.manual_seed(42)
np.random.seed(42)

print("Libraries imported successfully!")

In [ ]:
csv_path = "./data/adni/demographics/PTDEMOG.csv"
df = pd.read_csv(csv_path)

print(f"Demographics loaded: {df.shape}")
print(f"Columns: {list(df.columns)[:10]}...")

In [ ]:
def norm_codes_to_labels(s: pd.Series, mapping: dict) -> pd.Series:
    out = s.astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
    out = out.map(mapping)
    return out

gender_map = {"1":"male","2":"female","male":"male","female":"female","m":"male","f":"female"}
marry_map  = {"1":"married","2":"widowed","3":"divorced","4":"never_married","6":"domestic_partnership"}

def to_year(s):
    s = pd.to_numeric(s, errors="coerce")
    s = s.where((s >= 1900) & (s <= 2100))
    return s

print("Utility functions defined.")

In [ ]:

onset_cols = [c for c in ["PTCOGBEG","PTADBEG","PTADDX"] if c in df.columns]
for c in onset_cols:
    df[c] = to_year(df[c])

def row_min_nonnull(row):
    vals = [row[c] for c in onset_cols if pd.notna(row[c])]
    return min(vals) if vals else np.nan

df["YEAR_ONSET"] = df.apply(row_min_nonnull, axis=1) if onset_cols else np.nan
df["YEAR_ONSET"] = to_year(df["YEAR_ONSET"])

for c in ["PTDOBYY","PTEDUCAT"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

if "PTGENDER" in df.columns:
    df["PTGENDER"] = norm_codes_to_labels(df["PTGENDER"], gender_map)
if "PTMARRY" in df.columns:
    df["PTMARRY"]  = norm_codes_to_labels(df["PTMARRY"], marry_map)

print(f"\nData processed. Shape: {df.shape}")
print(f"Patients with YEAR_ONSET: {df['YEAR_ONSET'].notna().sum()}")

In [ ]:
biomarker_path = "./data/adni/demographics/UPENNBIOMK_ROCHE_ELECSYS_11Oct2025.csv"
df_csf = pd.read_csv(biomarker_path)

df_csf['VISCODE_NORMALIZED'] = df_csf['VISCODE2'].astype(str).str.strip()
df_csf = df_csf.dropna(subset=['ABETA42', 'TAU', 'PTAU'])

df_csf['TAU_ABETA42_RATIO'] = df_csf['TAU'] / (df_csf['ABETA42'] + 1e-6)
df_csf['PTAU_ABETA42_RATIO'] = df_csf['PTAU'] / (df_csf['ABETA42'] + 1e-6)
df_csf['PTAU_TAU_RATIO'] = df_csf['PTAU'] / (df_csf['TAU'] + 1e-6)

df['VISCODE_NORMALIZED'] = df['VISCODE2'].astype(str).str.strip().replace({'sc': 'bl', 'f': 'bl', 'nan': ''})
df = df.merge(
    df_csf[['RID', 'VISCODE_NORMALIZED', 'ABETA42', 'TAU', 'PTAU', 
            'TAU_ABETA42_RATIO', 'PTAU_ABETA42_RATIO', 'PTAU_TAU_RATIO']],
    on=['RID', 'VISCODE_NORMALIZED'],
    how='left'  # LEFT JOIN - mantener todas las visitas
)

df['HAS_CSF'] = df['ABETA42'].notna().astype(float)

print(f"✅ CSF merged (LEFT JOIN): {df['HAS_CSF'].sum():.0f}/{len(df)} visitas con CSF ({100*df['HAS_CSF'].mean():.1f}%)")

In [ ]:
pet_path = "./data/adni/demographics/All_Subjects_UCBERKELEY_AMY_6MM_11Oct2025.csv"
df_pet = pd.read_csv(pet_path, low_memory=False)

df_pet['VISCODE_NORMALIZED'] = df_pet['VISCODE'].astype(str).str.strip().replace({'sc': 'bl', 'f': 'bl', 'nan': ''})
df_pet = df_pet[['RID', 'VISCODE_NORMALIZED', 'CENTILOIDS', 'SUMMARY_SUVR', 'COMPOSITE_REF_SUVR']].copy()
df_pet.columns = ['RID', 'VISCODE_NORMALIZED', 'PET_CENTILOIDS', 'PET_SUVR', 'PET_COMPOSITE']
df_pet = df_pet.dropna(subset=['PET_CENTILOIDS', 'PET_SUVR'])

df = df.merge(df_pet, on=['RID', 'VISCODE_NORMALIZED'], how='left')
df['HAS_PET'] = df['PET_CENTILOIDS'].notna().astype(float)

print(f"✅ PET merged (LEFT JOIN): {df['HAS_PET'].sum():.0f}/{len(df)} visitas con PET ({100*df['HAS_PET'].mean():.1f}%)")

In [ ]:
mri_path = "./data/adni/demographics/All_Subjects_UCSFFSX7_11Oct2025.csv"
df_mri = pd.read_csv(mri_path, low_memory=False)

df_mri['VISCODE_NORMALIZED'] = df_mri['VISCODE2'].astype(str).str.strip().replace({'sc': 'bl', 'f': 'bl', 'nan': ''})
df_mri = df_mri[['RID', 'VISCODE_NORMALIZED', 'ST101SV', 'ST11SV', 'ST12SV', 
                 'ST4SV', 'ST5SV', 'ST17SV', 'ST18SV']].copy()
df_mri.columns = ['RID', 'VISCODE_NORMALIZED', 'MRI_eTIV', 'MRI_Vol1', 'MRI_Vol2', 
                  'MRI_Vol3', 'MRI_Vol4', 'MRI_Vol5', 'MRI_Vol6']
df_mri = df_mri.dropna(subset=['MRI_eTIV', 'MRI_Vol1'])

df = df.merge(df_mri, on=['RID', 'VISCODE_NORMALIZED'], how='left')
df['HAS_MRI'] = df['MRI_eTIV'].notna().astype(float)

df = df.drop(columns=['VISCODE_NORMALIZED'])

print(f"✅ MRI merged (LEFT JOIN): {df['HAS_MRI'].sum():.0f}/{len(df)} visitas con MRI ({100*df['HAS_MRI'].mean():.1f}%)")
print(f"\n📊 Dataset final: {len(df)} visitas con todos los biomarcadores opcionales")

In [ ]:

date_col = "EXAMDATE" if "EXAMDATE" in df.columns else ("VISDATE" if "VISDATE" in df.columns else None)
if not date_col:
    raise ValueError("No se encuentra columna de fecha (VISDATE/EXAMDATE)")

df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df["EXAM_YEAR"] = to_year(df[date_col].dt.year)

df["AGE_AT_VISIT"] = np.where(
    df["EXAM_YEAR"].notna() & df["PTDOBYY"].notna(),
    df["EXAM_YEAR"] - df["PTDOBYY"],
    np.nan
)

df["YEARS_TO_ONSET"] = np.where(
    df["YEAR_ONSET"].notna() & df["EXAM_YEAR"].notna(),
    df["YEAR_ONSET"] - df["EXAM_YEAR"],
    np.nan
)

df.loc[(df["YEARS_TO_ONSET"] < 0) & df["YEAR_ONSET"].notna(), "YEARS_TO_ONSET"] = np.nan
df.loc[df["YEARS_TO_ONSET"] > 50, "YEARS_TO_ONSET"] = np.nan

df["HAS_LABEL"] = df["YEARS_TO_ONSET"].notna()

print(f"\nVisits prepared: {len(df)}")
print(f"Labeled visits (with YEARS_TO_ONSET): {df['HAS_LABEL'].sum()}")
print(f"Using date column: {date_col}")

In [ ]:

csf_cols = ['ABETA42', 'TAU', 'PTAU', 'TAU_ABETA42_RATIO', 'PTAU_ABETA42_RATIO', 'PTAU_TAU_RATIO']
pet_cols = ['PET_CENTILOIDS', 'PET_SUVR', 'PET_COMPOSITE']
mri_cols = ['MRI_eTIV', 'MRI_Vol1', 'MRI_Vol2', 'MRI_Vol3', 'MRI_Vol4', 'MRI_Vol5', 'MRI_Vol6']
missing_indicators = ['HAS_CSF', 'HAS_PET', 'HAS_MRI']

num_cols = [c for c in ["AGE_AT_VISIT", "PTEDUCAT"] + csf_cols + pet_cols + mri_cols + missing_indicators if c in df.columns]

cat_cols = [c for c in ["PTGENDER", "PTMARRY"] if c in df.columns]

for c in csf_cols + pet_cols + mri_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

for c in ["AGE_AT_VISIT", "PTEDUCAT"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(df[c].median())

parts = []

if num_cols:
    scaler = StandardScaler()
    X_num = pd.DataFrame(
        scaler.fit_transform(df[num_cols]),
        columns=num_cols,
        index=df.index
    )
    parts.append(X_num)
    print(f"Numeric features ({len(num_cols)}):")
    print(f"  Demographics: 2")
    print(f"  CSF biomarkers: {len(csf_cols)}")
    print(f"  PET imaging: {len(pet_cols)}")
    print(f"  MRI volumetric: {len(mri_cols)}")
    print(f"  Missing indicators: {len(missing_indicators)}")

if cat_cols:
    X_cat = pd.get_dummies(df[cat_cols], prefix=cat_cols, drop_first=False, dtype=float)
    parts.append(X_cat)
    print(f"Categorical (one-hot): {X_cat.shape[1]} columns")

if not parts:
    raise ValueError("No features available")

X = pd.concat(parts, axis=1).astype(float)
X_clean = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)

print(f"\n✅ Feature matrix (ALL BIOMARKERS with missing handling): {X_clean.shape}")
print(f"Total features: {X_clean.shape[1]}")

In [ ]:

n_samples = X_clean.shape[0]
k = min(8, max(1, n_samples - 1))
n_neighbors = min(n_samples, k + 1)

if n_samples >= 2:
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric="euclidean")
    nbrs.fit(X_clean.values)
    _, idx = nbrs.kneighbors(X_clean.values)
    src_knn, dst_knn = [], []
    for i in range(idx.shape[0]):
        for j in idx[i, 1:]:
            src_knn.append(i)
            dst_knn.append(j)
    edge_knn = torch.tensor([src_knn, dst_knn], dtype=torch.long)
else:
    edge_knn = torch.empty((2, 0), dtype=torch.long)

tmp = df.reset_index()[["index", "RID", date_col]].dropna(subset=[date_col]).sort_values(["RID", date_col])
src_tmp, dst_tmp = [], []
for rid, g in tmp.groupby("RID"):
    ids = g["index"].tolist()
    for a, b in zip(ids[:-1], ids[1:]):
        src_tmp.append(a)
        dst_tmp.append(b)
edge_tmp = torch.tensor([src_tmp, dst_tmp], dtype=torch.long) if src_tmp else torch.empty((2,0), dtype=torch.long)

def undirected(e):
    return torch.cat([e, e.flip(0)], dim=1) if e.numel() else e

edges = []
if edge_knn.numel(): edges.append(undirected(edge_knn))
if edge_tmp.numel(): edges.append(undirected(edge_tmp))
edge_index = torch.cat(edges, dim=1) if edges else torch.empty((2,0), dtype=torch.long)
if edge_index.numel():
    edge_index = torch.unique(edge_index, dim=1)

print(f"\nGraph constructed:")
print(f"  Nodes: {len(df)}")
print(f"  kNN edges: {edge_knn.size(1)}")
print(f"  Temporal edges: {edge_tmp.size(1)}")
print(f"  Total edges (undirected, unique): {edge_index.size(1)}")

In [ ]:

df["USE_FOR_LABEL"] = False
if df["HAS_LABEL"].any():
    idx_last_pre = df.loc[df["HAS_LABEL"]].groupby("RID")[date_col].idxmax()
    df.loc[idx_last_pre, "USE_FOR_LABEL"] = True

rids_with_label = df.loc[df["USE_FOR_LABEL"], "RID"].dropna().unique()
rng = np.random.default_rng(42)
rng.shuffle(rids_with_label)
n_lab_rids = len(rids_with_label)

tr_n = max(1, int(0.7 * n_lab_rids))
va_n = max(0, int(0.15 * n_lab_rids))
if tr_n + va_n > max(0, n_lab_rids - 1):
    va_n = max(0, n_lab_rids - 1 - tr_n)

train_rids = set(rids_with_label[:tr_n])
val_rids   = set(rids_with_label[tr_n:tr_n+va_n])
test_rids  = set(rids_with_label[tr_n+va_n:])

node_split = np.full(len(df), "train", dtype=object)
node_rids = df["RID"].to_numpy()
node_split[np.isin(node_rids, list(val_rids))]  = "val"
node_split[np.isin(node_rids, list(test_rids))] = "test"

use_for_label = df["USE_FOR_LABEL"].to_numpy()
train_mask_np = (node_split == "train") & use_for_label
val_mask_np   = (node_split == "val")   & use_for_label
test_mask_np  = (node_split == "test")  & use_for_label

split_map = {"train":0, "val":1, "test":2}
split_idx = np.vectorize(split_map.get)(node_split)
src_np = edge_index[0].cpu().numpy()
dst_np = edge_index[1].cpu().numpy()
keep_edges = split_idx[src_np] == split_idx[dst_np]
edge_index = edge_index[:, torch.tensor(keep_edges)]

print(f"\nSplits created:")
print(f"  RIDs with labels: {n_lab_rids}")
print(f"  Train RIDs: {len(train_rids)}")
print(f"  Val RIDs: {len(val_rids)}")
print(f"  Test RIDs: {len(test_rids)}")
print(f"\n  Labeled nodes:")
print(f"    Train: {train_mask_np.sum()}")
print(f"    Val: {val_mask_np.sum()}")
print(f"    Test: {test_mask_np.sum()}")
print(f"\n  Edges (intra-split): {edge_index.size(1)}")

In [ ]:

y_full = df["YEARS_TO_ONSET"].astype(float)
y_mu  = float(y_full[train_mask_np].mean()) if train_mask_np.any() else 0.0
y_std = float(y_full[train_mask_np].std(ddof=0)) if train_mask_np.any() else 1.0
if not np.isfinite(y_std) or y_std == 0.0:
    y_std = 1.0

y_scaled = (y_full - y_mu) / y_std
y_t = torch.tensor(y_scaled.fillna(0).values, dtype=torch.float32)

x = torch.tensor(X_clean.values, dtype=torch.float32)
train_mask = torch.tensor(train_mask_np, dtype=torch.bool)
val_mask   = torch.tensor(val_mask_np,   dtype=torch.bool)
test_mask  = torch.tensor(test_mask_np,  dtype=torch.bool)

data = Data(x=x, edge_index=edge_index, y=y_t,
            train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

print(f"\nData object created:")
print(f"  x.shape: {data.x.shape}")
print(f"  edge_index.shape: {data.edge_index.shape}")
print(f"  y.shape: {data.y.shape}")
print(f"  Target scaling: mean={y_mu:.3f}, std={y_std:.3f}")

Y_MEAN_TRAIN, Y_STD_TRAIN = y_mu, y_std

In [ ]:

class GNNRegressor(nn.Module):
    def __init__(self, in_ch, hid=64, dropout=0.5):
        super().__init__()
        self.c1 = GCNConv(in_ch, hid)
        self.c2 = GCNConv(hid, 1)
        self.dropout = dropout
        
    def forward(self, x, edge_index):
        x = F.relu(self.c1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.c2(x, edge_index)
        return x.squeeze(-1)

device = torch.device("cpu")
model = GNNRegressor(in_ch=data.num_node_features, hid=64, dropout=0.5).to(device)
data = data.to(device)

print(f"\n🧠 Model created (REGULARIZED):")
print(f"  Input features: {data.num_node_features}")
print(f"  Hidden dim: 64")
print(f"  Dropout: 0.5 (increased for better regularization)")
print(f"  Architecture: GCNConv -> ReLU -> Dropout(0.5) -> GCNConv -> Output")
print(f"  Model parameters: {sum(p.numel() for p in model.parameters())}")

In [ ]:

class EarlyStopping:
    """Early stopping entre épocas 20-25"""
    def __init__(self, patience=5, min_epoch=20, min_delta=0.0):
        self.patience = patience
        self.min_epoch = min_epoch  # No activar antes de época 20
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_epoch = 0
        
    def __call__(self, epoch, val_loss):
        if epoch < self.min_epoch:
            if self.best_loss is None or val_loss < self.best_loss:
                self.best_loss = val_loss
                self.best_epoch = epoch
            return False
        
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                return True
        
        return False

early_stopping = EarlyStopping(patience=5, min_epoch=20, min_delta=1e-4)

print("✅ Early Stopping configured:")
print("  Patience: 5 epochs")
print("  Min epoch: 20 (starts checking after epoch 20)")
print("  Min delta: 1e-4")
print("  Expected stop: Between epochs 20-25")

In [ ]:

opt = torch.optim.AdamW(model.parameters(), lr=1e-2, weight_decay=1e-3)
loss_fn = nn.MSELoss()

history = {
    'epoch': [],
    'train_loss': [],
    'train_mae': [],
    'train_rmse': [],
    'val_mae': [],
    'val_rmse': [],
    'val_loss': [],
    'test_mae': [],
    'test_rmse': [],
    'overfitting_gap': []  # train_mae - val_mae
}

def eval_metrics(split="val"):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        mask = data.train_mask if split=="train" else (data.val_mask if split=="val" else data.test_mask)
        if not mask.any():
            return float("nan"), float("nan"), float("nan")
        mae_scaled = torch.mean(torch.abs(out[mask] - data.y[mask])).item()
        mse = loss_fn(out[mask], data.y[mask]).item()
        rmse_scaled = torch.sqrt(torch.tensor(mse)).item()
        return mae_scaled * Y_STD_TRAIN, rmse_scaled * Y_STD_TRAIN, mse

print("\n🚀 Starting training with Early Stopping + L2 Regularization...\n")
print("Regularization settings:")
print(f"  Dropout: 0.5")
print(f"  L2 (weight_decay): 1e-3")
print(f"  Learning rate: 1e-2")
print(f"  Early stopping: Active after epoch 20\n")

best_model_state = None

for ep in range(1, 101):
    model.train()
    opt.zero_grad()
    out = model(data.x, data.edge_index)
    
    if data.train_mask.any():
        loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
        if torch.isnan(loss) or torch.isinf(loss):
            raise RuntimeError("Loss NaN/Inf")
        loss.backward()
        opt.step()
        loss_val = loss.detach().item()
    else:
        loss_val = float("nan")
    
    tr_mae, tr_rmse, _ = eval_metrics("train")
    va_mae, va_rmse, va_loss = eval_metrics("val")
    te_mae, te_rmse, _ = eval_metrics("test")
    
    overfitting_gap = tr_mae - va_mae if not np.isnan(tr_mae) and not np.isnan(va_mae) else np.nan
    
    history['epoch'].append(ep)
    history['train_loss'].append(loss_val)
    history['train_mae'].append(tr_mae)
    history['train_rmse'].append(tr_rmse)
    history['val_mae'].append(va_mae)
    history['val_rmse'].append(va_rmse)
    history['val_loss'].append(va_loss)
    history['test_mae'].append(te_mae)
    history['test_rmse'].append(te_rmse)
    history['overfitting_gap'].append(overfitting_gap)
    
    if va_loss == early_stopping.best_loss:
        best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
    
    if ep % 5 == 0 or ep <= 25:
        gap_str = f"gap={overfitting_gap:+.3f}y" if not np.isnan(overfitting_gap) else "gap=N/A"
        print(
            f"ep {ep:03d} | train_loss {loss_val:.4f} "
            f"| TR MAE {tr_mae:.3f}y RMSE {tr_rmse:.3f}y "
            f"| VAL MAE {va_mae:.3f}y RMSE {va_rmse:.3f}y "
            f"| TEST MAE {te_mae:.3f}y RMSE {te_rmse:.3f}y | {gap_str}"
        )
    
    if early_stopping(ep, va_loss):
        print(f"\n⏸️  Early stopping triggered at epoch {ep}!")
        print(f"   Best validation loss: {early_stopping.best_loss:.4f} at epoch {early_stopping.best_epoch}")
        break

print("\n✅ Training completed!")
print(f"Final epoch: {ep}")
print(f"Best validation epoch: {early_stopping.best_epoch}")

if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"✅ Best model restored (epoch {early_stopping.best_epoch})")

import json
with open('training_history_regularized.json', 'w') as f:
    json.dump(history, f, indent=2)
print("✅ Training history saved to: training_history_regularized.json")

In [ ]:

model.eval()
with torch.no_grad():
    final_out = model(data.x, data.edge_index)

tr_mae, tr_rmse, _ = eval_metrics("train")
va_mae, va_rmse, _ = eval_metrics("val")
te_mae, te_rmse, _ = eval_metrics("test")

final_gap = tr_mae - va_mae

print("\n" + "="*70)
print("📊 REGULARIZED MODEL RESULTS (Early Stop + Dropout + L2)")
print("="*70)
print(f"Dataset: {len(df)} visits (ALL demographics)")
print(f"Features: {data.num_node_features} (demographics + CSF + PET + MRI + indicators)")
print(f"\nBiomarker availability:")
print(f"  CSF: {df['HAS_CSF'].sum():.0f} visits ({100*df['HAS_CSF'].mean():.1f}%)")
print(f"  PET: {df['HAS_PET'].sum():.0f} visits ({100*df['HAS_PET'].mean():.1f}%)")
print(f"  MRI: {df['HAS_MRI'].sum():.0f} visits ({100*df['HAS_MRI'].mean():.1f}%)")
print(f"\nRegularization applied:")
print(f"  ⏸️  Early stopping: Epoch {early_stopping.best_epoch} (stopped at {ep})")
print(f"  🎲 Dropout: 0.5")
print(f"  ⚖️  L2 weight decay: 1e-3")
print("\nPerformance:")
print(f"  TRAIN | MAE: {tr_mae:.3f} years | RMSE: {tr_rmse:.3f} years")
print(f"  VAL   | MAE: {va_mae:.3f} years | RMSE: {va_rmse:.3f} years")
print(f"  TEST  | MAE: {te_mae:.3f} years | RMSE: {te_rmse:.3f} years")
print(f"\n  📉 Overfitting gap (train-val MAE): {final_gap:+.3f} years")
print("="*70)

regularized_results = {
    'model': 'All Biomarkers + Early Stopping + Dropout(0.5) + L2(1e-3)',
    'features': data.num_node_features,
    'train_mae': tr_mae,
    'train_rmse': tr_rmse,
    'val_mae': va_mae,
    'val_rmse': va_rmse,
    'test_mae': te_mae,
    'test_rmse': te_rmse,
    'overfitting_gap': final_gap,
    'n_visits': len(df),
    'csf_coverage': float(df['HAS_CSF'].mean()),
    'pet_coverage': float(df['HAS_PET'].mean()),
    'mri_coverage': float(df['HAS_MRI'].mean()),
    'early_stop_epoch': early_stopping.best_epoch,
    'final_epoch': ep,
    'dropout': 0.5,
    'weight_decay': 1e-3
}

print("\n💡 Benefits of regularization:")
print("   ✓ Prevents overfitting (smaller gap between train/val)")
print("   ✓ Better generalization to test set")
print("   ✓ Stops training at optimal point (epochs 20-25)")
print("   ✓ Saves computation time")

In [ ]:

import json

with open('regularized_model_results.json', 'w') as f:
    json.dump(regularized_results, f, indent=2)

print("\n✅ Results saved to: regularized_model_results.json")
print("\n🎯 Next steps:")
print("   1. Compare with baseline model (AllBiomarkers.ipynb)")
print("   2. Check training curves for overfitting reduction")
print("   3. Verify early stopping worked correctly (epoch 20-25)")
print(f"\n📊 Summary:")
print(f"   - Stopped at epoch: {ep} (best: {early_stopping.best_epoch})")
print(f"   - Test MAE: {te_mae:.3f} years")
print(f"   - Overfitting gap: {final_gap:+.3f} years")